## Stats


#### imports


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from ast import literal_eval
import ast
from collections import Counter
import mlflow, hashlib, pathlib

DATA_PATH = Path(r"C:\Users\mzouicha\OneDrive - Amadeus Workplace\Desktop\STAGE\raop-pizza\data\processed\dataset_clean.csv")
df = pd.read_csv(DATA_PATH)
print(df.shape)
df.head(3)


(4040, 40)


,giver_username_if_known,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,post_was_edited,request_id,request_number_of_comments_at_retrieval,request_text,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,...,request_time_utc,request_text_original,request_text_edit_aware_original,request_title_original,request_text_length,request_text_word_count,request_text_edit_aware_length,request_text_edit_aware_word_count,request_title_length,request_title_word_count
0,NaN,0,1,False,t3_l25d7,0,hi i am in need of food for my 4 children we a...,hi i am in need of food for my 4 children we a...,request colorado springs help us please,0.0000,...,2011-10-05 21:10:07+00:00,Hi I am in need of food for my 4 children we a...,Hi I am in need of food for my 4 children we a...,Request Colorado Springs Help Us Please,316,67,316,67,39,6
1,NaN,2,5,False,t3_rcb83,0,i spent the last money i had on gas today. im ...,i spent the last money i had on gas today. im ...,"[request] california, no cash and i could use ...",501.1111,...,2012-03-25 04:13:44+00:00,I spent the last money I had on gas today. Im ...,I spent the last money I had on gas today. Im ...,"[Request] California, No cash and I could use ...",74,16,74,16,57,10
2,NaN,0,3,False,t3_lpu5j,0,my girlfriend decided it would be a good idea ...,my girlfriend decided it would be a good idea ...,"[request] hungry couple in dundee, scotland wo...",0.0000,...,2011-10-26 16:28:14+00:00,My girlfriend decided it would be a good idea ...,My girlfriend decided it would be a good idea ...,"[Request] Hungry couple in Dundee, Scotland wo...",277,59,277,59,66,10


##### 1- What is the prevalence for a requester to be given a pizza?

In [3]:
prevalence = df["requester_received_pizza"].mean()
print(f"Prevalence to be given a pizza is: {prevalence:.2%}")


Prevalence to be given a pizza is: 24.60%


##### 2- What are the top 10 subreddits the requesters have already posted in the most?

In [5]:

sample = df["requester_subreddits_at_request"].head(10)
for idx, val in sample.items():
    print(f"{idx}: {val!r}   (type: {type(val).__name__})")

print("\ntype pf columns")
print(df["requester_subreddits_at_request"].map(type).value_counts())

str_examples = df[
    df["requester_subreddits_at_request"].apply(
        lambda x: isinstance(x, str) and x.strip() not in {"", "[]"}
    )
]["requester_subreddits_at_request"].head(5)

print("\nExmaples")
for val in str_examples:
    print(val)


list_examples = df[
    df["requester_subreddits_at_request"].apply(lambda x: isinstance(x, list) and len(x) > 0)
]["requester_subreddits_at_request"].head(3)




0: '[]'   (type: str)
1: "['AskReddit', 'Eve', 'IAmA', 'MontereyBay', 'RandomKindness', 'RedditBiography', 'dubstep', 'gamecollecting', 'gaming', 'halo', 'i18n', 'techsupport']"   (type: str)
2: '[]'   (type: str)
3: "['AskReddit', 'DJs', 'IAmA', 'Random_Acts_Of_Pizza']"   (type: str)
4: "['GayBrosWeightLoss', 'RandomActsOfCookies', 'Random_Acts_Of_Pizza', 'gaybears', 'gaybros', 'gaybrosgonemild', 'gaymers', 'keto', 'loseit', 'pics', 'trees']"   (type: str)
5: "['AdviceAnimals', 'AskReddit', 'Autos', 'IAmA', 'RandomKindness', 'Random_Acts_Of_Pizza', 'TheFacebookDelusion', 'atheism', 'aww', 'cars', 'coupons', 'fffffffuuuuuuuuuuuu', 'funny', 'gaming', 'highdesert', 'loseit', 'pics', 'reddit.com', 'shittybattlestations', 'todayilearned', 'videos']"   (type: str)
6: '[]'   (type: str)
7: "['AskReddit', 'IAmA', 'Music', 'Random_Acts_Of_Pizza', 'fffffffuuuuuuuuuuuu', 'firstworldproblems', 'funny', 'gaming', 'gonewild', 'guns', 'knives', 'pics', 'relationships', 'religion', 'technology']"   (

In [ ]:



def parse_sub_list(x: str) -> list[str]:
    if x in ("[]", "", None):
        return []
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return []

df["subreddit_list"] = df["requester_subreddits_at_request"].apply(parse_sub_list)

# Subreddits occurence 
sub_counts = Counter(sub for subs in df["subreddit_list"] for sub in subs)
top10_df = pd.DataFrame(sub_counts.most_common(10), columns=["subreddit", "count"])
top10_df


,subreddit,count
0,AskReddit,2307
1,pics,1949
2,funny,1932
3,WTF,1515
4,IAmA,1507
5,gaming,1485
6,Random_Acts_Of_Pizza,1413
7,videos,1160
8,todayilearned,1103
9,AdviceAnimals,1021


##### 3- For each of these top 10 subreddits and for each month of data, how many of their active users (that have already posted in it) requested a pizza in the past 6 months?

In [7]:
df["month"] = pd.to_datetime(df["request_time_utc"]).dt.to_period("M")

top10_list = top10_df["subreddit"].tolist()

exploded = (
    df.explode("subreddit_list")
      .rename(columns={"subreddit_list": "subreddit"})
)
exploded = exploded[exploded["subreddit"].isin(top10_list)]

user_month = (
    exploded[["subreddit", "requester_username", "month"]]
        .drop_duplicates()
        .assign(active=1)
)


pivot = (
    user_month.pivot_table(
        index="month",
        columns="subreddit",
        values="active",
        aggfunc="sum",
        fill_value=0
    )
    .sort_index()
)


pivot_6m = (
    pivot
    .rolling(window=6, min_periods=1)
    .sum()
    .astype(int)
)

pivot_6m.head(12)   


C:\Users\mzouicha\AppData\Local\Temp\ipykernel_19972\2817717117.py:1: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["month"] = pd.to_datetime(df["request_time_utc"]).dt.to_period("M")


subreddit,AdviceAnimals,AskReddit,IAmA,Random_Acts_Of_Pizza,WTF,funny,gaming,pics,todayilearned,videos
month,,,,,,,,,,
2011-02,1,1,1,1,1,0,0,1,0,1
2011-05,1,3,3,1,2,1,1,2,2,2
2011-06,6,97,62,10,50,61,64,76,45,44
2011-07,21,163,104,34,93,118,116,143,70,76
2011-08,41,275,184,63,157,204,189,254,126,140
2011-09,56,351,238,77,201,267,239,326,162,180
2011-10,67,410,271,92,229,319,276,381,191,208
2011-11,77,464,308,197,265,360,313,433,212,232
2011-12,86,422,284,306,255,347,288,407,193,215


##### 4- What is the average time between two consecutive requests for each month, only for months with at least 50 requests?

In [8]:
df_sorted = df.sort_values("request_time_utc")
df_sorted["gap_hours"] = (
    pd.to_datetime(df_sorted["request_time_utc"])
    .diff()
    .dt.total_seconds()
    / 3600
)

monthly = df_sorted.groupby("month")
monthly_sizes = monthly.size()

mask = monthly_sizes[monthly_sizes >= 50].index
monthly_gap = (
    monthly["gap_hours"].mean()
    .loc[mask]
    .round(2)
)

monthly_gap.head()


month
2011-06    4.06
2011-07    3.08
2011-08    3.15
2011-09    4.50
2011-10    4.89
Freq: M, Name: gap_hours, dtype: float64

#### 5- Petit Recap

In [8]:
print("Prevalence :", f"{prevalence:.2%}")
print("\nTop 10 subreddits :")
display(top10_df)

print("\nExemple de tableau fenêtres 6 mois (top 5 lignes) :")
display(pivot_6m.head())

print("\nTemps moyen entre requêtes (heures) pour les mois actifs :")
display(monthly_gap.head())


Prevalence : 24.60%

Top 10 subreddits :


,subreddit,count
0,AskReddit,2307
1,pics,1949
2,funny,1932
3,WTF,1515
4,IAmA,1507
5,gaming,1485
6,Random_Acts_Of_Pizza,1413
7,videos,1160
8,todayilearned,1103
9,AdviceAnimals,1021



Exemple de tableau fenêtres 6 mois (top 5 lignes) :


subreddit,AdviceAnimals,AskReddit,IAmA,Random_Acts_Of_Pizza,WTF,funny,gaming,pics,todayilearned,videos
month,,,,,,,,,,
2011-02,1,1,1,1,1,0,0,1,0,1
2011-05,1,3,3,1,2,1,1,2,2,2
2011-06,6,97,62,10,50,61,64,76,45,44
2011-07,21,163,104,34,93,118,116,143,70,76
2011-08,41,275,184,63,157,204,189,254,126,140



Temps moyen entre requêtes (heures) pour les mois actifs :


month
2011-06    4.06
2011-07    3.08
2011-08    3.15
2011-09    4.50
2011-10    4.89
Freq: M, Name: gap_hours, dtype: float64

##### log in MLFlow


In [ ]:


root_mlruns = Path.cwd().parent / "mlruns"   # si ton notebook est dans notebooks/
mlflow.set_tracking_uri(f"file:{root_mlruns}")
mlflow.set_experiment("raop_stats")


with mlflow.start_run(run_name="stats_v1"):
    mlflow.log_param("dataset_md5", hashlib.md5(open(DATA_PATH, "rb").read()).hexdigest())
    mlflow.log_metric("prevalence", prevalence)
    # on peut enregistrer top10, pivot_6m, monthly_gap comme artifacts CSV/Parquet
    top10_df.to_csv("top10_subs.csv", index=False)
    pivot_6m.to_csv("pivot_6m.csv")
    monthly_gap.to_csv("monthly_gap.csv")
    mlflow.log_artifact("top10_subs.csv")
    mlflow.log_artifact("pivot_6m.csv")
    mlflow.log_artifact("monthly_gap.csv")


In [ ]:


# On veut pointer sur le dossier mlruns à la racine du projet
root_mlruns = Path.cwd().parent / "mlruns"   # si ton notebook est dans notebooks/
mlflow.set_tracking_uri(f"file:{root_mlruns}")
mlflow.set_experiment("raop_stats")
print("MLflow tracking URI ⇒", mlflow.get_tracking_uri())


2025/06/18 21:34:33 INFO mlflow.tracking.fluent: Experiment with name 'raop_stats' does not exist. Creating a new experiment.


MLflow tracking URI ⇒ file:c:\Users\mzouicha\OneDrive - Amadeus Workplace\Desktop\STAGE\raop-pizza\mlruns
